In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
# @IMPORT-MERGE
import os
os.environ["MPLCONFIGDIR"] = "/tmp"
import numpy as np
import pandas as pd
import itertools
import cv2
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.run.sigproc_v2.sigproc_v2_result import sig_from_df_filter
from plaster.run.plots import plots, plots_dev
from plaster.run.sigproc_v2.synth import Synth
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.utils import data
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.utils import data
from plaster.tools.calibration.calibration import Calibration
from plaster.tools.ipynb_helpers.displays import hd
z = zplots.setup()

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/abbe5_1t")
run = job.runs[0]

# Calibration

In [ ]:
# Show Calibration information
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    psfs = run.sigproc_v2.params.calibration.psfs(ch_i=ch_i)

    with z(_noaxes=True):
        display("Below:")
        with z(_cols=6):
            n_zs = psfs.shape[0]
            for z_i in range(0, n_zs // 2):
                plots_dev.plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z)

        display("Most in Focus:")
        z_i = n_zs // 2
        plots_dev.plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z, _noaxes=True, _notools=True)

        display("Above:")
        with z(_cols=6):
            for z_i in range(n_zs // 2 + 1, n_zs):
                plots_dev.plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z)
                
    # Show the Calibration illumination balance
    illum = np.array(run.sigproc_v2.params.calibration[f"regional_illumination_balance.instrument_channel[{ch_i}]"])
    z.im(1.0 / illum, f_title="Illuminati")

# Quality

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    qdf = run.ims_import.qualities()
    quality = qdf[qdf.channel_i == ch_i].sort_values(["quality"])
    z.hist(quality.quality, _size_x=800, _size_y=150, f_title=f"Quality distribution channel {ch_i}")

    row_iz = utils.ispace(0, len(qdf), 3)

    # COMBINE all images for common percentile calculations
    ims = np.concatenate([
        run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i].flatten()
        for row in qdf.iloc[row_iz].itertuples()
    ])
    bot, top = np.percentile(ims, (50, 99))

    # SHOW example of worst, median, and best all using the same cspan
    with z(_cols=3, _cspan=(bot, top)):
        names = ("worst", "median", "best")
        for name, row in zip(names, qdf.iloc[row_iz].itertuples()):
            z.im(run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i], f_title=f"Channel: {ch_i} {name}")            

# Alignment

In [ ]:
field_df = run.sigproc_v2.fields().copy()
field_df["align"] = np.sqrt(field_df.aln_x**2 + field_df.aln_y**2)
alignment = field_df.groupby("field_i").align.max().values
z.cols(alignment, f_x_axis_label="field_i", f_y_axis_label="n_pixels", f_title="Max. alignment dist.")

# SNR

In [ ]:
df = run.sigproc_v2.fields__n_peaks__peaks__radmat()

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    snr = run.sigproc_v2.snr()[:, ch_i, :]
    _df = df.groupby(["field_i", "cycle_i"]).snr.mean().reset_index()
    snr_by_field_cycle = (
        pd.pivot_table(
            _df, values="snr", index="field_i", columns=["cycle_i"]
        )
        .reset_index()
        .rename_axis(None, axis=1)
        .drop(columns="field_i")
    ).values

    with z(_cols=2):
        z.hist(
            snr, _bins=(0, 100, 100),
            f_y_axis_label="count", f_x_axis_label="SNR",
            f_title=f"SNR distribution ch_i={ch_i}"
        )
        z.im_clus(
            snr_by_field_cycle,
            f_y_axis_label="field_i", f_x_axis_label="cycle_i",
            f_title=f"Mean SNR by field and cycle ch_i={ch_i}"
        )
    


# Signal

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    sig = run.sigproc_v2.sig()[:, ch_i, :]
    _df = df.groupby(["field_i", "cycle_i"]).signal.mean().reset_index()
    sig_by_field_cycle = (
        pd.pivot_table(
            _df, values="signal", index="field_i", columns=["cycle_i"]
        )
        .reset_index()
        .rename_axis(None, axis=1)
        .drop(columns="field_i")
    ).values

    top = np.percentile(sig, 97)

    with z(_cols=2):
        z.hist(
            sig, _bins=(0, top, 100),
            f_y_axis_label="count", f_x_axis_label="Signal",
            f_title=f"Signal distribution ch_i={ch_i}"
        )
        z.im_clus(
            sig_by_field_cycle,
            f_y_axis_label="field_i", f_x_axis_label="cycle_i",
            f_title=f"Mean Sig by field and cycle ch_i={ch_i}",
            _cspan=(0, top),
        )

    z.im_clus(sig, _cspan=(0, top), f_title=f"radmat sample ch_i={ch_i}")

In [ ]:
# Histograms by cycle
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    sig = run.sigproc_v2.sig()[:, ch_i, 0].flatten()
    center = np.median(sig)  #5_000  # This is a reference line, move it around as you want
    n_cycles = run.sigproc_v2.n_cycles
    max_x = np.percentile(sig, 97)
    bins = np.linspace(0, max_x, 100)
    _hist, _ = np.histogram(sig, bins=bins)
    max_y = np.max(_hist)
    with z(_cols=3, _size=250, _bins=bins, _range=(0, max_x, 0, max_y*1.5)):
        for cy_i in range(n_cycles):
            sig = run.sigproc_v2.sig()[:, ch_i, cy_i].flatten()
            with z(_merge=True, f_title=f"cy_i={cy_i}"):
                z.hist(sig)
                z.line(x=[center, center], y=[0, max_y], color="red")


# Image Visualization

In [ ]:
# View a field, channel, cycle with roll-over properties for debugging

# Change this line to select a different (field, channel, cycle) to view:
field_channel_cycle = (0, 0, 0)

plots_dev.sigproc_v2_im(run, *field_channel_cycle, _full=True)


# Wizards

In [ ]:
plots.wizard_scat_df(run, channel_i=0)

In [ ]:
plots.wizard_xy_df(run, channel_i=0)

In [ ]:
plots.wizard_raw_images(run, show_circles=False, peak_i_square=True, square_radius=7)